In [5]:
import numpy as np
# ^^^ pyforest auto-imports - don't write above this line
import pyforest
import struct
import skimage.transform
import tensorflow as tf
import datetime

# Dependency imports
from tensorflow.keras import datasets, layers, models
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.wrappers import scikit_learn
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from PIL import Image
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder

# Notebook Extentions
%load_ext tensorboard

## CNN Model

#### CNN Model Prep

In [6]:
X_train = np.load('data/japanese_X_train.npz')['arr_0']
X_val = np.load('data/japanese_X_validation.npz')['arr_0']
y_train = np.load('data/japanese_y_train.npz')['arr_0']
y_val = np.load('data/japanese_X_validation.npz')['arr_0']

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
# Image dimensions
img_row, img_col = 64, 64

# Classes
japanese_class = 3

In [8]:
# Default shape for CNN is 'channels_last', this makes sure everything is in the right order

if K.image_data_format() == 'channels_first':
    japanese_X_train = X_train.reshape(X_train.shape[0], 1, img_row, img_col)
    japanese_X_val = X_val.reshape(X_val.shape[0], 1, img_row, img_col)
    japanese_shape = (1, img_row, img_col) 
else:
    japanese_X_train = X_train.reshape(X_train.shape[0], img_row, img_col, 1)
    japanese_X_val = X_val.reshape(X_val.shape[0], img_row, img_col, 1)
    japanese_shape = (img_row, img_col, 1)

In [11]:
cnn_model = models.Sequential()

cnn_model.add(layers.Conv2D(32, (3, 3), activation = 'relu', padding = 'same', input_shape = japanese_shape, data_format = 'channels_last'))
cnn_model.add(layers.MaxPooling2D((2, 2)))
cnn_model.add(layers.Conv2D(32, (3, 3), activation = 'relu', padding = 'same'))
cnn_model.add(Dropout(0.2))

cnn_model.add(layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same'))
cnn_model.add(layers.MaxPooling2D((2, 2)))
cnn_model.add(layers.Conv2D(128, (3, 3), activation = 'relu', padding = 'same'))
cnn_model.add(Dropout(0.2))

cnn_model.add(Flatten())
cnn_model.add(Dense(256, activation = 'relu'))
cnn_model.add(Dropout(0.2))
cnn_model.add(Dense(japanese_class, activation = 'softmax'))

In [12]:
cnn_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 64, 64, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
dropout_3 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 16, 16, 128)      

In [13]:
# Callbacks
japanese_es = EarlyStopping(patience = 5, verbose = 1, restore_best_weights = True)
japanese_lr = ReduceLROnPlateau(factor = 0.5, patience = 3, verbose = 1)

# Tensorboard Callback
japanese_log_dir = 'logs/CNN/' + datetime.datetime.now().strftime('Y%m%d-%H%M%S')
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = japanese_log_dir, histogram_freq = 1)

#### Adding Variation with ImageDataGenerator
- Tensorflow's ImageDataGenerator can augment images to create new vesions of the data, essentially artificially inserting variation into the data
- Images can be rotated, zoomed, and flipped
- This variation is used to avoid overfitting on the training set
- Following augmentation is for a 2D CNN

In [ ]:
datagen = ImageDataGenerator(rotation_range=15,zoom_range=0.2)
datagen.fit(japanese_X_train)

cnn_model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

cnn_results = cnn_model.fit(datagen.flow(japanese_X_train, japanese_y_train, batch_size = 32),
          epochs = 30, callbacks = (japanese_es, japanese_lr, tensorboard_callback), validation_data = (japanese_X_val, japanese_y_val))